In [1]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [2]:
from emlp.mlp_jax import MLP,EMLP,LinearBNSwish
from emlp.datasets_jax import Fr
import haiku as hk
import jax.numpy as jnp
import jax
import optax

In [3]:
from emlp.equivariant_subspaces_jax import T,Scalar,Matrix,Vector,Quad,repsize
from emlp.groups import SO,O,Trivial,Lorentz,O13,SO13,SO13p
from emlp.mlp_jax import EMLP,LieLinear
import itertools
import numpy as np
import torch
from emlp.datasets_jax import Inertia,Fr,ParticleInteraction

In [4]:
import torch
from torch.utils.data import DataLoader
from slax.utils.utils import LoaderTo,cosLr, islice, export,FixedNumpySeed
from slax.tuning.study import train_trial
from slax.datasetup.datasets import split_dataset
from slax.model_trainers.classifier import Regressor
from functools import partial
import torch.nn as nn
#repmiddle = 100*T(0)+30*T(1)+10*T(2)+3*T(3)#+1*T(4)

def makeTrainer(*,dataset=Fr,network=EMLP,num_epochs=500,ndata=1000+1000,seed=2020,aug=False,
                bs=500,lr=1e-2,device='cuda',split={'train':-1,'test':1000},
                net_config={'num_layers':3,'group':SO(3)},
                trainer_config={'log_dir':None,'log_args':{'minPeriod':.02}},save=False):

    # Prep the datasets splits, model, and dataloaders
    with FixedNumpySeed(seed):
        datasets = split_dataset(dataset(ndata),splits=split)
    device = torch.device(device)
    def model(x,is_training): 
        return network(datasets['train'].rep_in,datasets['train'].rep_out,**net_config)(x,is_training)
    dataloaders = {k:LoaderTo(DataLoader(v,batch_size=bs,shuffle=(k=='train'),
                num_workers=0,pin_memory=False)) for k,v in datasets.items()}
    dataloaders['Train'] = dataloaders['train']
    opt_constr = optax.adam(lr)
    return Regressor(model,dataloaders,opt_constr,**trainer_config)

In [5]:
#trainer = makeTrainer(network=EMLP,lr=3e-3)
trainer = makeTrainer(network=MLP,lr=1e-3,aug=False)
trainer.train(500)

INFO:root:Initializing network


KeyboardInterrupt: 

In [6]:
%debug

> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/tensorboardX/summary.py(117)scalar()
    115     name = _clean_tag(name)
    116     scalar = make_np(scalar)
--> 117     assert(scalar.squeeze().ndim == 0), 'scalar should be 0D'
    118     scalar = float(scalar)
    119     return Summary(value=[Summary.Value(tag=name, simple_value=scalar)])

ipdb> print(scalar)
None
ipdb> q


In [18]:
x = ds.X.numpy()[:10]
#net = hk.without_apply_rng(hk.transform(net_fn))
net = hk.without_apply_rng(hk.transform_with_state(net_fn))
rng = jax.random.PRNGKey(42)
# The `init` function now returns parameters **and** state. State contains
# anything that was created using `hk.set_state`. The structure is the same as
# params (e.g. it is a per-module mapping of named values).
params, state = net.init(rng, x, is_training=True)
opt = optax.adam(1e-3)
opt_state = opt.init(params)

DEBUG:root:[2T(1) @ d=3, 102T(0)+34T(1)+11T(2)+3T(3) @ d=3, 102T(0)+34T(1)+11T(2)+3T(3) @ d=3, 102T(0)+34T(1)+11T(2)+3T(3) @ d=3]
DEBUG:root:BiW components:2592 dim:600 shape:(432, 6) rep:300T(1)+68T(2)+22T(3)+6T(4) @ d=3
DEBUG:root:BiW components:165888 dim:65568 shape:(432, 384) rep:15300T(0)+8568T(1)+3928T(2)+1504T(3)+325T(4)+66T(5)+9T(6) @ d=3
DEBUG:root:BiW components:165888 dim:65568 shape:(432, 384) rep:15300T(0)+8568T(1)+3928T(2)+1504T(3)+325T(4)+66T(5)+9T(6) @ d=3
DEBUG:root:bilinear in shape: (10, 6)
DEBUG:root:bilinear out shape: (10, 432)
DEBUG:root:linear in shape: (10, 6)
DEBUG:root:linear out shape:(10, 432)
DEBUG:root:bilinear in shape: (10, 384)
DEBUG:root:bilinear out shape: (10, 432)
DEBUG:root:linear in shape: (10, 384)
DEBUG:root:linear out shape:(10, 432)
DEBUG:root:bilinear in shape: (10, 384)
DEBUG:root:bilinear out shape: (10, 432)
DEBUG:root:linear in shape: (10, 384)
DEBUG:root:linear out shape:(10, 432)
DEBUG:root:linear in shape: (10, 384)
DEBUG:root:linear

bn input shape (10, 432)
bn output shape (10, 432)
bn input shape (10, 432)
bn output shape (10, 432)
bn input shape (10, 432)
bn output shape (10, 432)


In [19]:
logging.getLogger().setLevel(logging.INFO)

In [20]:
# The apply function now takes both params **and** state. Additionally it will
# return updated values for state. In the resnet example this will be the
# updated values for moving averages used in the batch norm layers.
logits, state = net.apply(params, state, x)

bn input shape (10, 432)
bn output shape (10, 432)
bn input shape (10, 432)
bn output shape (10, 432)
bn input shape (10, 432)
bn output shape (10, 432)


In [16]:
state

FlatMapping({
  'emlp/~/tensor_mask_bn/~/mean_ema': FlatMapping({
                                        'counter': DeviceArray(1, dtype=int32),
                                        'hidden': DeviceArray([-2.72169262e-02,  3.35316695e-02, -2.94104172e-03,
                                                                6.48176298e-03, -3.12723741e-02, -2.12527718e-02,
                                                               -1.36882742e-03, -1.15396166e-02, -2.69303899e-02,
                                                                2.59548817e-02, -8.30877572e-03, -4.91703255e-03,
                                                               -3.85290794e-02,  1.22720916e-02,  1.94078311e-02,
                                                               -2.23876797e-02,  1.66982170e-02, -2.91648041e-03,
                                                               -2.40416378e-02,  2.69001815e-02, -1.66822281e-02,
                                                        

In [11]:
%debug

> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py(4326)_canonicalize_tuple_index()
   4324   if len_without_none > arr_ndim:
   4325     msg = "Too many indices for array: {} non-None/Ellipsis indices for dim {}."
-> 4326     raise IndexError(msg.format(len_without_none, arr_ndim))
   4327   ellipses = (i for i, elt in enumerate(idx) if elt is Ellipsis)
   4328   ellipsis_index = next(ellipses, None)

ipdb> up
> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py(4047)_index_to_gather()
   4045 def _index_to_gather(x_shape, idx):
   4046   # Remove ellipses and add trailing slice(None)s.
-> 4047   idx = _canonicalize_tuple_index(len(x_shape), idx)
   4048 
   4049   # Check for advanced indexing:

ipdb> up
> /home/marc/anaconda3/envs/default/lib/python3.7/site-packages/jax/_src/numpy/lax_numpy.py(3963)_gather()
   3961 def _gather(arr, treedef, static_idx, dynamic_idx):
   3962   idx = _merge_static_and_dyn

In [ ]:
import numpy as np
np.random.randn(3,4,2).sum((0,1))

In [7]:
import numpy as np
np.arange(10).dtype

dtype('int64')

In [2]:
bs = 256
c=512
key = random.PRNGKey(0)
x = random.normal(key,(bs,c))
W = random.normal(key,(bs,c,c))
W2 = random.normal(key,(c,c))
z = random.normal(key,(c,))

In [16]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
import numpy as np
from functools import partial
from jax.ops import index, index_add, index_update
import haiku as hk
from functools import partial

In [3]:
W2@x[0];

In [34]:
class Linear(hk.Module):
    """Linear module."""
    def __init__(self,cin,cout,name = None):
        super().__init__(name=name)
        w_init = hk.initializers.TruncatedNormal(1. / np.sqrt(cin))
        self.w = hk.get_parameter("w", shape=[cout, cin], init=w_init)
        self.b = hk.get_parameter("b", shape=[cout], init=jnp.zeros)

    def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
        """Computes a linear transform of the input."""
        
        return self.w@x + self.b

In [35]:
net = hk.without_apply_rng(hk.transform(lambda a: Linear(c,3)(a)))

In [36]:
key = hk.PRNGSequence(42)
params = net.init(next(key), x[0])

In [37]:
nn = jit(vmap(partial(net.apply,params)))

In [38]:
nn(x).shape

(256, 3)

In [39]:
z.shape

(512,)

In [15]:
params['linear']['w'].shape

(512, 3)

In [40]:
x.sigmoid()

AttributeError: 'DeviceArray' object has no attribute 'sigmoid'

In [12]:
import jax
from parallax import Module, Parameter, ParamInit
import jax.nn.initializers as init
class Dense(Module):
    # All parameter-holders are explicitly declared.
    weight : Parameter
    bias : Parameter

    # Setup replace __init__ and creates shapes and binds lazy initializers.
    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = ParamInit((out_size, in_size), init.xavier_normal())
        self.bias = ParamInit((out_size,), init.normal())


    # Forward is just like standard pytorch.
    def forward(self, input):
        return self.weight @ input + self.bias

    # Hook for pretty printing
    def extra_repr(self):
        return "%d, %d"%(self.weight.shape[1], self.weight.shape[0])

class Dropout(Module):
    # Arbitrary constants allowed.
    #rate : float
    def __init__(self, rate):
        super().__init__()
        self.rate = rate

    def forward(self, input):
        # RNG state is use-once or split. Attached to tree.
        state = self.rng

        if self.mode == "train":
            keep = jax.random.bernoulli(state, self.rate, input.shape)
            return jax.numpy.where(keep, input / self.rate, 0)
        else:
            return input

class BinaryNetwork(Module):
    # No difference between modules and parameters
    dense1 : Dense
    dense2 : Dense
    dense3 : Dense
    dropout : Dropout

    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.dense1 = Dense(input_size, hidden_size)
        self.dense2 = Dense(hidden_size, hidden_size)
        self.dense3 = Dense(hidden_size, 1)
        self.dropout = Dropout(0.2)

    def forward(self, input):

        # Standard usage works out of the box.
        x = jax.numpy.tanh(self.dense1(input))

        # Stochastic modules (have random seed already)
        x = self.dropout(x)

        # Shared params / recurrence only requires split to change RNG
        x = jax.numpy.tanh(self.dense2(x))
        x = jax.numpy.tanh(self.dense2(x))

        return jax.nn.sigmoid(self.dense3(jax.numpy.tanh(x)))[0]

# Setup param tree -> declarative, immutable
layer = BinaryNetwork(5, 10)
print(layer)
print(layer.dense1)

# Initialize parameters -> stateful, hidden
rng = jax.random.PRNGKey(0)
layer = layer.initialized(rng)
print(layer)
print(layer.dense1)

initial_loss = None
for i in range(10):
    # Thread state through parameters -> functor, hidden
    rng, iter_rng = jax.random.split(rng)
    layer = layer.new_state(iter_rng, mode="train")
    
    # Jax style grad compute -> tree-shaped immutable
    x = jax.numpy.zeros(5)
    loss = layer(x)
    if initial_loss is None:
        initial_loss = loss
    print(loss)
    grad = layer.grad(x)
    
    # Grad Update -> tree-shaped
    layer = jax.tree_util.tree_multimap(lambda p, g: p - 0.3 * g, layer, grad)

Exception: Field rate was not declared in <class '__main__.Dropout'> or ancestors!

In [38]:
ord('J')-ord('A')

9

In [37]:
'j'<'h'

False

In [47]:
def num(s,memo):
    ls = len(s)
    if s not in memo:
        if ls==0: 
            memo[s] = 1
        elif ls==1:
            if s[0]<1 or s[0]>9: 
                memo[s] = 0
            else:
                memo[s]= 1
        else:
            if (s[-2]*10+s[-1]>26) or ((s[-2]*10+s[-1])<10):
                memo[s]=0
            else:
                memo[s]= num(s[:-2],memo)+num(s[:-1],memo)*num(s[-1:],memo)
    return memo[s]

In [48]:
num((1,2),{})

() 1
(1,) 1
(2,) 1
(1, 2) 2


2